---
# <span style="color:purple"> CLEANSING DATA </span>
---

## <span style="color:purple"> Inisiasi </span>

In [2]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd

host = '127.0.0.1'
port = "3306"
user = 'root'
password = 'ugm2018'
database = 'gdelt2'

table_name = 'events_SN'
fips_country_code = 'SN'
country_code = 'SGP'
# BRN, KHM, LAO, IDN, MYS, VNM, THA, SGP, PHL, MMR

conn_str = 'mysql+pymysql://{}:{}@{}:{}/{}'.format(user, password, host, port, database);           
engine = create_engine(conn_str)

def run(sql):
    df = pd.read_sql_query(sql,engine)
    return df

def execute(sql):
    return engine.execute(statement=sql)


### <span style="color:purple"> Cleansing Strategy </span>
- Observe Events by year
- Make sure date in range of 2003-2017
- Filter by Actor1CountryCode or Actor2CountryCode
- Deduplicate SOURCEURL (contained http/https)
- For null SOURCEURL and non http source, deduplicate news by theirs similiarities (AvgTone, ActionGeoLat, ActionGeoLong, GoldSteinScale, EventCode)

****

### a. Get Total News (Before Cleansing)
___

In [3]:
total_news = run("""
SELECT         
    count(GLOBALEVENTID) as count 
FROM {} 
""".format(table_name))

print('Total News :', total_news['count'][0])

Total News : 1913966


### b. Make sure date in range of 2003-2017
___

#### 1. Preview

In [4]:
out_of_range_counter = run("""
select count(GLOBALEVENTID) as count from {} where Year < 2003 or Year > 2017
""".format(table_name))

print('Out of Range :', out_of_range_counter['count'][0])


Out of Range : 285891


#### 2. Execute Deletion

In [5]:
execute("""
delete from {} where Year < 2003 or Year > 2017
""".format(table_name))

total_news = run("""
SELECT         
    count(GLOBALEVENTID) as count 
FROM {} 
""".format(table_name))

print('Total News Now:', total_news['count'][0])

Total News Now: 1628075


### c. Filter by Actor1CountryCode or Actor2CountryCode
---

#### 1. Preview

In [ ]:
data = run("""
select GLOBALEVENTID, SOURCEURL, Actor1CountryCode, Actor2CountryCode 
from {} WHERE Actor1CountryCode <> '{}' AND Actor2CountryCode <> '{}'
""".format(table_name, country_code, country_code))

data

#### 2. Execute Deletion

In [6]:
execute("""
delete from {} WHERE Actor1CountryCode <> '{}' AND Actor2CountryCode <> '{}'
""".format(table_name, country_code, country_code))

total_news = run("""
SELECT         
    count(GLOBALEVENTID) as count 
FROM {} 
""".format(table_name))

print('Total News Now:', total_news['count'][0])

Total News Now: 872095


### d. Deduplicate by SOURCEURL (contained http or https )
----

#### 1. Preview

In [ ]:
duplicateURL = run("""
SELECT t1.GLOBALEVENTID, t1.SOURCEURL, t1.NumArticles,
  ROW_NUMBER() OVER(PARTITION BY t1.SOURCEURL ORDER BY t1.SOURCEURL, t1.NumArticles DESC) AS DuplicateCount
  FROM {} t1
  WHERE t1.SOURCEURL like 'http://%%' OR t1.SOURCEURL LIKE 'https://%%'  
  
  """.format(table_name))

duplicateURL

#### 2. Execute Deletion

In [ ]:
execute("""
 DELETE FROM {}
 WHERE GLOBALEVENTID in (
 (SELECT t2.GLOBALEVENTID
 FROM
 (SELECT t1.GLOBALEVENTID, t1.SOURCEURL,  
  ROW_NUMBER() OVER(PARTITION BY t1.SOURCEURL ORDER BY t1.SOURCEURL, t1.NumArticles DESC) AS DuplicateCount
  FROM events_BX t1
  WHERE t1.SOURCEURL like 'http://%%' OR t1.SOURCEURL LIKE 'https://%%'
    ) t2
  WHERE t2.DuplicateCount > 1)
  )
 """)

total_news = run("""
SELECT         
    count(GLOBALEVENTID) as count 
FROM {} 
""".format(table_name,table_name))

print('Total News Now:', total_news['count'][0])

### d. Deduplicate by SOURCEURL (other than contained http or https )
----
Key: Have similarities in SQLDATE, AvgTone, ActionGeoLat, ActionGeoLong, GoldSteinScale, EventCode

#### 1. Preview

In [ ]:
duplicateURL2 = run("""
SELECT t1.GLOBALEVENTID, t1.NumArticles, t1.SOURCEURL, t1.SQLDATE,t1.AvgTone, t1.ActionGeo_Lat, t1.ActionGeo_Long,
  ROW_NUMBER() OVER(PARTITION BY t1.SQLDATE,t1.AvgTone, t1.ActionGeo_Lat, t1.ActionGeo_Long
  ORDER BY t1.SQLDATE, t1.AvgTone, t1.NumArticles DESC) AS DuplicateCount
  FROM {} t1  
  WHERE t1.SOURCEURL not like '%%http://%%' OR t1.SOURCEURL not LIKE '%%https://%%'
  """.format(table_name))

duplicateURL2

#### 2. Execute Deletion

In [ ]:
execute("""
 DELETE FROM {}
 WHERE GLOBALEVENTID in (
 (SELECT t2.GLOBALEVENTID
 FROM
 (SELECT t1.GLOBALEVENTID,
  ROW_NUMBER() OVER(PARTITION BY t1.SQLDATE, t1.AvgTone, t1.ActionGeo_Lat, t1.ActionGeo_Long, t1.SOURCEURL
  ORDER BY t1.SQLDATE, t1.AvgTone, t1.NumArticles DESC) AS DuplicateCount
  FROM {} t1 WHERE t1.SOURCEURL not like 'http://%%' OR t1.SOURCEURL not LIKE 'https://%%') t2
  WHERE t2.DuplicateCount > 1)
  )
 """.format(table_name, table_name))

total_news = run("""
SELECT         
    count(GLOBALEVENTID) as count 
FROM {} 
""".format(table_name,table_name))

print('Total News Now:', total_news['count'][0])

### f. Get Total News (After Cleansing)
___

In [ ]:
total_news = run("""
SELECT         
    count(GLOBALEVENTID) as count 
FROM {} 
""".format(table_name))

print('Total News :', total_news['count'][0])

---
# <span style="color:purple"> REKAP DATA </span>
---

#### 1. Overview Result

In [ ]:
overviewResult = run("""
SELECT 
    Year, 
    count(a.GLOBALEVENTID) as `Total Events`     
FROM {} a GROUP BY Year ORDER BY Year
  """.format(table_name))

overviewResult

#### 2. Rekap Event

In [ ]:

event = run("""
SELECT         
    Year, 
    CONCAT(EventCode,'. ',(SELECT description FROM CAMEO_events_code WHERE code = EventCode)) as Description,
    count(GLOBALEVENTID) as Jumlah
FROM {} 
WHERE Year > 2002
GROUP BY EventCode, Year
  """.format(table_name))

event.pivot(index='Description', columns='Year', values='Jumlah')

#### 2. Annual AvgTone Data

In [ ]:
rekap_akhir = run("""
SELECT 
    Year, 
    count(a.GLOBALEVENTID) as `Total Events`, 
    sum(CASE WHEN AvgTone > 0 THEN 1 ELSE 0 END) as `Total +AvgTone`,
    sum(CASE WHEN AvgTone < 0 THEN 1 ELSE 0 END) as `Total -AvgTone`,    
    sum(CASE WHEN AvgTone = 0 THEN 1 ELSE 0 END) as `Total Netral AvgTone`,    
    avg(AvgTone) as `AvgTone`,
    avg(CASE WHEN AvgTone > 0 THEN AvgTone ELSE 0 END) as `+AvgTone`,
    avg(CASE WHEN AvgTone < 0 THEN AvgTone ELSE 0 END) as `-AvgTone`,    
    (sum(AvgTone * NumArticles)/sum(NumArticles)) as `Weighted AvgTone`    
FROM {} a GROUP BY Year ORDER BY Year
  """.format(table_name))

# max(case when AvgTone >= 0 then AvgTone end) max_positive,
# max(case when AvgTone < 0 then AvgTone end) max_negative
rekap_akhir.to_csv('rekap' + fips_country_code + '.csv')
rekap_akhir

In [ ]:
execute("""
DELETE FROM events_SN
""")